# Implementation of is_seperable() function
## Step 1: Determine if the state is PSD

In [111]:
using LinearAlgebra
using Ket
using TensorCore

In [112]:
tol = eps()^(3/4)
tol

1.8189894035458565e-12

In [113]:
function has_positive_eigenvals(X)
    for i in eigvals(X)
        if !(i >= tol)
            print(i)
            return false
        end
    end
    return true
end

has_positive_eigenvals (generic function with 1 method)

In [114]:
# Source https://math.stackexchange.com/questions/357980/how-to-generate-random-symmetric-positive-definite-matrices-using-matlab
function generate_random_symmetric_psd(d)
    A = rand(Float64,(d,d))
    A = 0.5*(A+A')
    A = A + d * I
    A = A / tr(A)
    return A
end

generate_random_symmetric_psd (generic function with 1 method)

In [115]:
d = 2
A = kron(generate_random_symmetric_psd(d),generate_random_symmetric_psd(d))
B = generate_random_symmetric_psd(d)


2×2 Matrix{Float64}:
 0.508919   0.0787877
 0.0787877  0.491081

In [116]:
println(isposdef(A))
println(isposdef(B))


true
true


In [117]:
function IsSeperable(X)

    lX = length(X);
    rX = rank(X);
    X = X/tr(X);
    sep = -1;
    dim = round(Int, sqrt(lX))
    println(A)
    println(dim)
    println(Ket.ppt_mixture(A,[dim]))
    if(!isposdef(X))
        error("Not positive semidefinite, can't determine separability")
    end


    
end

IsSeperable (generic function with 1 method)

In [118]:
A = kron([[1,0] [0,1]],[[1,0] [0,1]])
Ket.ppt_mixture(A,[4])

(-0.333333328888889, [0 0 0 0; 0 0 0 0; 0 0 0 0; 0 0 0 0])

In [119]:
function is_ppt(X)
    lX = length(X);

    dim = round(Int, sqrt(lX))
    return isposdef(Ket.partial_transpose(X,[dim,dim]))
end

is_ppt (generic function with 1 method)

In [120]:
is_ppt(B)

LoadError: ArgumentError: Subsystems are not equally-sized, please specify sizes.